In [152]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (ensemble, preprocessing, tree)
from sklearn.metrics import (auc,confusion_matrix, roc_auc_score, roc_curve)
from sklearn import model_selection
from sklearn.model_selection import(train_test_split, StratifiedKFold)
from yellowbrick.classifier import ( ConfusionMatrix, ROCAUC)
from yellowbrick.model_selection import( LearningCurve)

# Gather data

In [153]:
arq_excel = 'titanic3.xls'

In [154]:
df_titanic_orig = pd.read_excel(arq_excel)

In [155]:
df_titanic_orig

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


# Clean data

In [156]:
df_titanic_orig.dtypes

pclass         int64
survived       int64
name          object
sex           object
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object

In [157]:
df_titanic_orig.shape

(1309, 14)

In [158]:
df_titanic_orig.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [159]:
df_titanic_orig.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [160]:
df_titanic_orig.isnull().mean()

pclass       0.000000
survived     0.000000
name         0.000000
sex          0.000000
age          0.200917
sibsp        0.000000
parch        0.000000
ticket       0.000000
fare         0.000764
cabin        0.774637
embarked     0.001528
boat         0.628724
body         0.907563
home.dest    0.430863
dtype: float64

In [161]:
df_titanic_orig.isnull().sum(axis=1)

0       1
1       1
2       2
3       1
4       2
       ..
1304    3
1305    5
1306    3
1307    4
1308    4
Length: 1309, dtype: int64

In [162]:
mask = df_titanic_orig.isnull().any(axis=1)

In [163]:
mask.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [164]:
df_titanic_orig[mask].body

0         NaN
1         NaN
2         NaN
3       135.0
4         NaN
        ...  
1304    328.0
1305      NaN
1306    304.0
1307      NaN
1308      NaN
Name: body, Length: 1309, dtype: float64

In [165]:
df_titanic_orig.sex.value_counts(dropna=False)

sex
male      843
female    466
Name: count, dtype: int64

In [166]:
df_titanic_orig.embarked.value_counts(dropna=False)

embarked
S      914
C      270
Q      123
NaN      2
Name: count, dtype: int64

# Features

In [167]:
name = df_titanic_orig.name

In [168]:
name.head(5)

0                      Allen, Miss. Elisabeth Walton
1                     Allison, Master. Hudson Trevor
2                       Allison, Miss. Helen Loraine
3               Allison, Mr. Hudson Joshua Creighton
4    Allison, Mrs. Hudson J C (Bessie Waldo Daniels)
Name: name, dtype: object

In [169]:
df_titanic_orig =df_titanic_orig.drop(columns=["name", "ticket", "body", "boat", "cabin", "home.dest"])

In [170]:
df_titanic_orig = pd.get_dummies(df_titanic_orig)

In [171]:
df_titanic_orig.columns

Index(['pclass', 'survived', 'age', 'sibsp', 'parch', 'fare', 'sex_female',
       'sex_male', 'embarked_C', 'embarked_Q', 'embarked_S'],
      dtype='object')

In [172]:
df_titanic_orig = df_titanic_orig.drop(columns=["sex_male", "embarked_C"])

In [173]:
y = df_titanic_orig.survived

In [174]:
X = df_titanic_orig.drop(columns="survived")

# Sample Data

In [175]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y, test_size = 0.3, random_state=42)

# Input data

In [176]:
from sklearn.experimental import (enable_iterative_imputer)

In [177]:
from sklearn import impute

In [178]:
df_titanic_orig.columns

Index(['pclass', 'survived', 'age', 'sibsp', 'parch', 'fare', 'sex_female',
       'embarked_Q', 'embarked_S'],
      dtype='object')

In [179]:
num_cols = ["pclass", "age", "sibsp", "parch", "fare", "sex_female"]

In [180]:
imputer = impute.IterativeImputer()

In [181]:
imputed = imputer.fit_transform(X_train[num_cols])

In [182]:
X_train.loc[:, num_cols] = imputed

/var/folders/wb/tfh3xb1s2tnd4gf6y_vkzcq40000gn/T/ipykernel_40073/3767055828.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [183]:
imputed = imputer.transform(X_test[num_cols])

In [184]:
X_test.loc[:, num_cols] = imputed

/var/folders/wb/tfh3xb1s2tnd4gf6y_vkzcq40000gn/T/ipykernel_40073/3891742130.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0.
 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1.
 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1.
 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 

## impute with the median

In [79]:
meds = X_train.median()

In [80]:
X_train = X_train.fillna(meds)

In [82]:
X_test = X_test.fillna(meds)

# Normalize Data

In [185]:
cols = "pclass,age,sibsp,fare".split(",")

In [186]:
X_train_to_scale = X_train[cols]

In [187]:
sca = preprocessing.StandardScaler()

In [188]:
X_train_scaled = sca.fit_transform(X_train_to_scale)  # Escale apenas essas colunas


In [145]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=cols)